In [ ]:
titles = []
for text in df_war.page_title:
    titles.append(text)

In [ ]:
texts_lower_case = [text.lower() for text in titles]

In [ ]:
import re
only_words_text = [re.sub('[\W_]+', ' ', text) for text in texts_lower_case]

In [ ]:
texts_splitted = [text.split() for text in only_words_text]

In [ ]:
#
"""Create list of stopwords"""
stopwords = ["и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", "его",
"но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", "вот", "от", "меня", "еще",
"нет", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже", "или", "ни", "быть", "был",
"него", "до", "вас", "нибудь", "опять", "уж", "вам", "ведь", "там", "потом", "себя", "ничего", "ей", "может", "они",
"тут", "где", "есть", "надо", "ней", "для", "мы", "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего",
"раз", "тоже", "себе", "под", "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", "совсем",
"ним", "здесь", "этом", "один", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", "куда", "зачем", "всех",
"никогда", "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", "над", "больше", "тот", "через", "эти",
"нас", "про", "всего", "них", "какая", "много", "разве", "три", "эту", "моя", "впрочем", "хорошо", "свою", "этой",
"перед", "иногда", "лучше", "чуть", "том", "нельзя", "такой", "им", "более", "всегда", "конечно", "всю", "между", 'иза']

In [ ]:
without_sw = [[word for word in text if word not in stopwords] for text in texts_splitted]

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

"""Function to convert words to initial form"""
def normal_rus(w):
    global morph
    return morph.parse(w)[0].normal_form

In [ ]:
from tqdm import tqdm
lemmatized_text = [[normal_rus(word) for word in text] for text in tqdm(without_sw)]

In [ ]:
from gensim import corpora

dictionary = corpora.Dictionary(lemmatized_text)
dict(dictionary)

In [ ]:
bows = [dictionary.doc2bow(text) for text in lemmatized_text]

In [ ]:
from gensim.models import TfidfModel
tfidf = TfidfModel(bows[0:100])

In [ ]:
from gensim import similarities
cos_sim = similarities.MatrixSimilarity(tfidf[bows[0:100]])

In [ ]:
from scipy.cluster import hierarchy
clusters = hierarchy.linkage(cos_sim, method='ward')

In [ ]:
# преобразуем в датафрейм
cos_sim_df = pd.DataFrame(list(cos_sim))
# # # добавним название текстов
cos_sim_df.columns = titles[0:100]
cos_sim_df.index = titles[0:100]

In [ ]:
from scipy.cluster import hierarchy
clusters = hierarchy.linkage(cos_sim_df, method='ward')

dendrogram = hierarchy.dendrogram(clusters,
                                  leaf_font_size=11,
                                  labels=list(cos_sim_df.index),
                                  orientation="left",
                                  truncate_mode='level', p=20)